In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!unzip /kaggle/input/dogs-vs-cats/train.zip
!unzip /kaggle/input/dogs-vs-cats/test1.zip

In [ ]:
train_dir = './train'
test_dir = './test1'
train_files = os.listdir(train_dir)
test_files = os.listdir(test_dir)

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import torch
import torch.nn as nn
import cv2
import matplotlib.pyplot as plt
import torchvision
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from torchvision import transforms
import copy
from tqdm import tqdm
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F

%matplotlib inline

In [ ]:
class CatDogDataset(Dataset):
    def __init__(self, file_list, dir, mode='train', transform = None):
        self.file_list = file_list
        self.dir = dir
        self.mode= mode
        self.transform = transform
        if self.mode == 'train':
            if 'dog' in self.file_list[0]:
                self.label = 1
            else:
                self.label = 0
            
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, idx):
        img = Image.open(os.path.join(self.dir, self.file_list[idx]))
        
        if self.transform:
            img = self.transform(img)
        if self.mode == 'train':
            img = img.numpy()
            return img.astype('float32'), self.label
        else:
            img = img.numpy()
            return img.astype('float32'), self.file_list[idx]

IMG_SIZE = (128, 128)
NORMALIZE_MEAN = (0.5, 0.5, 0.5)
NORMALIZE_STD = (0.5, 0.5, 0.5)

data_transform = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.ToTensor(),
])

In [ ]:
cat_files = [tf for tf in train_files if 'cat' in tf]
dog_files = [tf for tf in train_files if 'dog' in tf]

cats = CatDogDataset(cat_files[:int(len(cat_files)*0.9)], train_dir, transform = data_transform)
dogs = CatDogDataset(dog_files[:int(len(cat_files)*0.9)], train_dir, transform = data_transform)

cat_val = CatDogDataset(cat_files[(int(len(cat_files)*0.9) + 1):], train_dir, transform = data_transform)
dog_val = CatDogDataset(dog_files[(int(len(cat_files)*0.9) + 1):], train_dir, transform = data_transform)

catdogs = ConcatDataset([cats, dogs])
dataloader = DataLoader(catdogs, batch_size = 32, shuffle=True, num_workers=2)

val_catdogs = ConcatDataset([cat_val, dog_val])
val_dataloader = DataLoader(val_catdogs, batch_size = 32, shuffle=True, num_workers=2)

In [ ]:
print("Number of cat files: ",len(cat_files))
print("Number of dog files: ",len(dog_files))

In [ ]:
#Test if dataloader worked
samples, labels = iter(dataloader).next()
plt.figure(figsize=(16,24))
grid_imgs = torchvision.utils.make_grid(samples[:24])
np_grid_imgs = grid_imgs.numpy()
# in tensor, image is (batch, width, height), so you have to transpose it to (width, height, batch) in numpy to show it.
plt.imshow(np.transpose(np_grid_imgs, (1,2,0)))

In [ ]:
!pip install vit-pytorch

In [ ]:
from vit_pytorch import ViT

In [ ]:
class ViTModel(nn.Module):
    
    def __init__(self):
        super(ViTModel,self).__init__()
        self.vit = ViT(
                    image_size = IMG_SIZE,
                    patch_size = 16,
                    num_classes = 2,
                    dim = 1024,
                    depth = 6,
                    heads = 16,
                    mlp_dim = 512,
                    dropout = 0.1,
                    emb_dropout = 0.1
                )
    
    def forward(self,x):
        
        x = self.vit(x)
        x = F.softmax(x,dim=-1)
        return x

In [ ]:
#Shape of each input is (32, 3, 128, 128) -> batch size 32
device = 'cuda'
model = ViTModel()
model = model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.00001)

In [ ]:
epochs = 50
device = 'cuda'



model.train()
for epoch in range(epochs):
    

    for samples, labels in tqdm(dataloader):
        samples, labels = samples.to(device), labels.to(device)
        
        optimizer.zero_grad()
        output = model(samples)
        loss = criterion(output,labels)
        
        loss.backward()
        optimizer.step()
        
        
       
    #Finding accuracy at the end of an epoch
    total_loss = 0
    pred_list = torch.tensor([]).to(device)
    true_list = torch.tensor([]).to(device)
    for samples, labels in val_dataloader:
        samples, labels = samples.to(device), labels.to(device)
        
        output = model(samples)
        total_loss += criterion(output,labels).item()
        pred = torch.argmax(output, dim=1)
        pred_list = torch.cat((pred_list,pred),0)
        true_list = torch.cat((true_list,labels),0)
    
    correct = pred_list.eq(true_list)
    acc = torch.mean(correct.float())
    print('[Epoch {}/{}] -> Train Loss: {:.4f}, val_accuracy: {:.3f}'.format(epoch+1, epochs, total_loss, acc))
        